In [1]:
!pip install replicate
!pip install pandas
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.3 MB/s eta 0:00:00


In [3]:
import replicate
import pandas as pd
from PIL import Image
import os
from datetime import datetime, timedelta
from google.colab import userdata, files

api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [6]:
model = replicate.Client(api_token=api_token)
print(model)

In [12]:
#nutrition database
nutrition_db = {
    "rice": {"calories": 130, "protein": 2.7, "vitamins": "B1"},
    "spinach": {"calories": 23, "protein": 2.9, "vitamins": "A, C, K"},
    "fish": {"calories": 206, "protein": 22, "vitamins": "D, B12"},
    "egg": {"calories": 68, "protein": 6, "vitamins": "B12, D"},  # per egg (~50g)
    "chicken": {"calories": 165, "protein": 31, "vitamins": "B3, B6"},
    "broccoli": {"calories": 35, "protein": 2.8, "vitamins": "C, K, A"},
    "potato": {"calories": 77, "protein": 2, "vitamins": "C, B6"},
    "tempeh": {"calories": 193, "protein": 19, "vitamins": "B2, B3"},
    "tofu": {"calories": 76, "protein": 8, "vitamins": "B1"},
    "carrot": {"calories": 41, "protein": 0.9, "vitamins": "A, K"},
    "red kidney beans": {"calories": 127, "protein": 8.7, "vitamins": "B1, B9"},
    "avocado": {"calories": 160, "protein": 2, "vitamins": "E, K, C"},
    "banana": {"calories": 89, "protein": 1.1, "vitamins": "B6, C"},
    "red spinach": {"calories": 22, "protein": 2.2, "vitamins": "A, C"},
    "cabbage": {"calories": 25, "protein": 1.3, "vitamins": "C, K"},
    "beef": {"calories": 250, "protein": 26, "vitamins": "B12, B3"},
    "milk": {"calories": 61, "protein": 3.2, "vitamins": "D, B2, B12"},
    "cheese": {"calories": 403, "protein": 25, "vitamins": "A, B12"},
    "mung beans": {"calories": 105, "protein": 7, "vitamins": "B1, B9"},
    "corn": {"calories": 86, "protein": 3.2, "vitamins": "B1, B5"},
    "cassava": {"calories": 112, "protein": 1.4, "vitamins": "C, B6"},
    "sweet potato": {"calories": 86, "protein": 1.6, "vitamins": "A, C, B6"},
    "kangkung": {"calories": 19, "protein": 2.6, "vitamins": "A, C, K"},
    "long beans": {"calories": 47, "protein": 2.8, "vitamins": "A, C"},
    "soybean": {"calories": 147, "protein": 12.3, "vitamins": "B1, B9"},
    "peanut": {"calories": 567, "protein": 25.8, "vitamins": "E, B3"},  # use sparingly due to high calories
    "papaya": {"calories": 43, "protein": 0.5, "vitamins": "C, A"},
    "mango": {"calories": 60, "protein": 0.8, "vitamins": "C, A, B6"},
    "guava": {"calories": 68, "protein": 2.6, "vitamins": "C, A"},
    "tempeh goreng": {"calories": 225, "protein": 18, "vitamins": "B2, B3"},  # fried tempeh
    "tahu goreng": {"calories": 115, "protein": 9, "vitamins": "B1"},  # fried tofu
    "chicken liver": {"calories": 116, "protein": 16.9, "vitamins": "A, B12, B2"},
    "anchovy": {"calories": 131, "protein": 20.4, "vitamins": "D, B3"},
    "sardine": {"calories": 208, "protein": 24.6, "vitamins": "D, B12"},
    "oatmeal": {"calories": 68, "protein": 2.4, "vitamins": "B1, B5"},
    "green beans": {"calories": 31, "protein": 1.8, "vitamins": "C, K"},
    "jackfruit": {"calories": 95, "protein": 1.7, "vitamins": "C, A"},
    "durian": {"calories": 147, "protein": 1.5, "vitamins": "C, B1"},  # use sparingly due to high calories
    "salted egg": {"calories": 140, "protein": 10, "vitamins": "B12, D"},  # per egg (~70g)
    "kolplay": {"calories": 28, "protein": 1.4, "vitamins": "C, K"}  # kolplay or water spinach
}
nutrition_db = {
    "rice": {"calories": 130, "protein": 2.7, "vitamins": "B1"},
    "spinach": {"calories": 23, "protein": 2.9, "vitamins": "A, C, K"},
    "fish": {"calories": 206, "protein": 22, "vitamins": "D, B12"},
    "egg": {"calories": 68, "protein": 6, "vitamins": "B12, D"},  # per egg (~50g)
    "chicken": {"calories": 165, "protein": 31, "vitamins": "B3, B6"},
    "broccoli": {"calories": 35, "protein": 2.8, "vitamins": "C, K, A"},
    "potato": {"calories": 77, "protein": 2, "vitamins": "C, B6"},
    "tempeh": {"calories": 193, "protein": 19, "vitamins": "B2, B3"},
    "tofu": {"calories": 76, "protein": 8, "vitamins": "B1"},
    "carrot": {"calories": 41, "protein": 0.9, "vitamins": "A, K"},
    "red kidney beans": {"calories": 127, "protein": 8.7, "vitamins": "B1, B9"},
    "avocado": {"calories": 160, "protein": 2, "vitamins": "E, K, C"},
    "banana": {"calories": 89, "protein": 1.1, "vitamins": "B6, C"},
    "red spinach": {"calories": 22, "protein": 2.2, "vitamins": "A, C"},
    "cabbage": {"calories": 25, "protein": 1.3, "vitamins": "C, K"},
    "beef": {"calories": 250, "protein": 26, "vitamins": "B12, B3"},
    "milk": {"calories": 61, "protein": 3.2, "vitamins": "D, B2, B12"},
    "cheese": {"calories": 403, "protein": 25, "vitamins": "A, B12"},
    "mung beans": {"calories": 105, "protein": 7, "vitamins": "B1, B9"},
    "corn": {"calories": 86, "protein": 3.2, "vitamins": "B1, B5"},
    "cassava": {"calories": 112, "protein": 1.4, "vitamins": "C, B6"},
    "sweet potato": {"calories": 86, "protein": 1.6, "vitamins": "A, C, B6"},
    "kangkung": {"calories": 19, "protein": 2.6, "vitamins": "A, C, K"},
    "long beans": {"calories": 47, "protein": 2.8, "vitamins": "A, C"},
    "soybean": {"calories": 147, "protein": 12.3, "vitamins": "B1, B9"},
    "peanut": {"calories": 567, "protein": 25.8, "vitamins": "E, B3"},  # use sparingly due to high calories
    "papaya": {"calories": 43, "protein": 0.5, "vitamins": "C, A"},
    "mango": {"calories": 60, "protein": 0.8, "vitamins": "C, A, B6"},
    "guava": {"calories": 68, "protein": 2.6, "vitamins": "C, A"},
    "tempeh goreng": {"calories": 225, "protein": 18, "vitamins": "B2, B3"},  # fried tempeh
    "tahu goreng": {"calories": 115, "protein": 9, "vitamins": "B1"},  # fried tofu
    "chicken liver": {"calories": 116, "protein": 16.9, "vitamins": "A, B12, B2"},
    "anchovy": {"calories": 131, "protein": 20.4, "vitamins": "D, B3"},
    "sardine": {"calories": 208, "protein": 24.6, "vitamins": "D, B12"},
    "oatmeal": {"calories": 68, "protein": 2.4, "vitamins": "B1, B5"},
    "green beans": {"calories": 31, "protein": 1.8, "vitamins": "C, K"},
    "jackfruit": {"calories": 95, "protein": 1.7, "vitamins": "C, A"},
    "durian": {"calories": 147, "protein": 1.5, "vitamins": "C, B1"},  # use sparingly due to high calories
    "salted egg": {"calories": 140, "protein": 10, "vitamins": "B12, D"},  # per egg (~70g)
    "kolplay": {"calories": 28, "protein": 1.4, "vitamins": "C, K"}  # kolplay or water spinach
}
print(nutrition_db)

{'rice': {'calories': 130, 'protein': 2.7, 'vitamins': 'B1'}, 'spinach': {'calories': 23, 'protein': 2.9, 'vitamins': 'A, C, K'}, 'fish': {'calories': 206, 'protein': 22, 'vitamins': 'D, B12'}, 'egg': {'calories': 68, 'protein': 6, 'vitamins': 'B12, D'}, 'chicken': {'calories': 165, 'protein': 31, 'vitamins': 'B3, B6'}, 'broccoli': {'calories': 35, 'protein': 2.8, 'vitamins': 'C, K, A'}, 'potato': {'calories': 77, 'protein': 2, 'vitamins': 'C, B6'}, 'tempeh': {'calories': 193, 'protein': 19, 'vitamins': 'B2, B3'}, 'tofu': {'calories': 76, 'protein': 8, 'vitamins': 'B1'}, 'carrot': {'calories': 41, 'protein': 0.9, 'vitamins': 'A, K'}, 'red kidney beans': {'calories': 127, 'protein': 8.7, 'vitamins': 'B1, B9'}, 'avocado': {'calories': 160, 'protein': 2, 'vitamins': 'E, K, C'}, 'banana': {'calories': 89, 'protein': 1.1, 'vitamins': 'B6, C'}, 'red spinach': {'calories': 22, 'protein': 2.2, 'vitamins': 'A, C'}, 'cabbage': {'calories': 25, 'protein': 1.3, 'vitamins': 'C, K'}, 'beef': {'calor

In [11]:
# Function to analyze food image
def analyze_food_image(image_path):
    try:
        # Read image
        with open(image_path, "rb") as image_file:
            # Send image to Granite Vision model
            prompt = (
                "Analyze this food image. Identify the types of food in the image "
                "and provide an estimate of nutritional content (calories, protein, vitamins) for each food type. "
                "Also provide recommendations to improve nutritional value for toddlers."
            )
            output = model.run(
                "ibm-granite/granite-vision-3.2-2b",
                input={
                    "image": image_file,
                    "prompt": prompt,
                    "max_tokens": 200
                }
            )
            return output
    except Exception as e:
        return f"Error in image analysis: {str(e)}"

# Function to parse model output and calculate nutrition
def parse_nutrition_output(output):
    # Simple parsing example (adjust based on model output format)
    identified_foods = []
    recommendations = []

    # Join list elements into a single string and split into lines
    output_text = "".join(output)
    lines = output_text.split("\n")
    for line in lines:
        line = line.lower().strip()
        if "food identification" in line:
            for food in nutrition_db.keys():
                if food in line:
                    identified_foods.append(food)
        if "recommendation" in line:
            recommendations.append(line.replace("recommendation:", "").strip())

    # Calculate total nutrition
    total_nutrition = {"calories": 0, "protein": 0, "vitamins": []}
    for food in identified_foods:
        if food in nutrition_db:
            total_nutrition["calories"] += nutrition_db[food]["calories"]
            total_nutrition["protein"] += nutrition_db[food]["protein"]
            total_nutrition["vitamins"].append(nutrition_db[food]["vitamins"])

    return identified_foods, total_nutrition, recommendations

# Function to save weekly report
def save_weekly_report(date, foods, nutrition, recommendations):
    report_data = {
        "Date": date,
        "Foods": ", ".join(foods),
        "Calories (kcal)": nutrition["calories"],
        "Protein (g)": nutrition["protein"],
        "Vitamins": ", ".join(nutrition["vitamins"]),
        "Recommendations": "; ".join(recommendations)
    }

    # Save to CSV
    report_file = "weekly_nutrition_report.csv"
    df = pd.DataFrame([report_data])
    if os.path.exists(report_file):
        df_existing = pd.read_csv(report_file)
        df = pd.concat([df_existing, df], ignore_index=True)
    df.to_csv(report_file, index=False)
    return report_file

# Function to check stunting risk
def check_stunting_risk(report_file):
    try:
        df = pd.read_csv(report_file)
        # Assume daily nutritional requirements for toddlers (simple example)
        min_protein = 20  # grams per day
        min_calories = 800  # kcal per day

        # Get data from the last 7 days
        last_week = df[df["Date"] >= (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")]
        avg_protein = last_week["Protein (g)"].mean()
        avg_calories = last_week["Calories (kcal)"].mean()

        risk = []
        if avg_protein < min_protein:
            risk.append("Protein deficiency (average: {:.1f} g, minimum: {} g)".format(avg_protein, min_protein))
        if avg_calories < min_calories:
            risk.append("Calorie deficiency (average: {:.1f} kcal, minimum: {} kcal)".format(avg_calories, min_calories))

        return risk if risk else ["No stunting risk detected based on weekly data."]
    except Exception as e:
        return [f"Error in stunting risk analysis: {str(e)}"]

# Main program
def main():
    # Upload image
    print("Upload food image (PNG/JPEG format):")
    uploaded = files.upload()
    if not uploaded:
        print("No image uploaded.")
        return

    image_path = list(uploaded.keys())[0]

    # Analyze image
    print("\nAnalyzing image...")
    output = analyze_food_image(image_path)
    print("Model output:", output)

    # Parse results
    foods, nutrition, recommendations = parse_nutrition_output(output)
    print("\nAnalysis Results:")
    print(f"Detected foods: {', '.join(foods)}")
    print(f"Total Calories: {nutrition['calories']} kcal")
    print(f"Total Protein: {nutrition['protein']} g")
    print(f"Vitamins: {', '.join(nutrition['vitamins'])}")
    print(f"Recommendations: {'; '.join(recommendations)}")

    # Save report
    current_date = datetime.now().strftime("%Y-%m-%d")
    report_file = save_weekly_report(current_date, foods, nutrition, recommendations)
    print(f"\nReport saved to: {report_file}")

    # Check stunting risk
    risks = check_stunting_risk(report_file)
    print("\nStunting Risk Analysis:")
    for risk in risks:
        print(f"- {risk}")

if __name__ == "__main__":
    main()

Upload food image (PNG/JPEG format):


Saving images.jpg to images (1).jpg

Analyzing image...
Model output: ['', 'The image', ' contains a variety of food items that are commonly found in a to', "ddler's meal. The rice serves as a source", " of carbohydrates, which are the body's", ' primary energy source. The tomato slices provide vitamins', ' A and C, as well as some minerals like potass', 'ium and lycopene. The green peas contribute to', ' the fiber and vitamin content, while the fried egg adds', ' protein and healthy fats. The corn kernels are also a', ' good source of carbohydrates and fiber.\n\n', 'For a toddler, it is important to ensure a balance', ' of macronutrients, including carbohydrates', ', proteins, and fats. The rice provides carbo', 'hydrates, the tomato and corn kernels offer', ' fiber and vitamins, the peas are rich in vit', 'amins and minerals, and the egg is a good source', ' of protein. The']

Analysis Results:
Detected foods: 
Total Calories: 0 kcal
Total Protein: 0 g
Vitamins: 
Recommendations: 

R